In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [4]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

df = pd.read_csv("./data/titanic.csv")
# titanic.csv 파일에서 데이터를 읽어와 DataFrame으로 저장합니다.
tool = PythonAstREPLTool(locals={"df": df})
# PythonAstREPLTool을 사용하여 로컬 변수 'df'를 포함하는 환경을 생성합니다.
tool.invoke("df")
# 'df' DataFrame에서 'Fare' 열의 평균 값을 계산합니다.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
hello = """
print("Hello, world!")

def add(a, b):
    return a + b

print(add(30, 40))

import pandas as pd

df = pd.read_csv("./data/titanic.csv")
df.head()
"""

In [9]:
tool = PythonAstREPLTool(locals={"df": df})
# PythonAstREPLTool을 사용하여 로컬 변수 'df'를 포함하는 환경을 생성합니다.
tool.invoke(hello)

Hello, world!
70


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 데이터 로드

pandas를 활용하여 csv 파일을 DataFrame 으로 로드합니다.


In [7]:
import pandas as pd

df = pd.read_csv("data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Pandas DataFrame Agent


In [36]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.callbacks.base import BaseCallbackHandler


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)


# # 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(
        temperature=0,
        model_name="gpt-3.5-turbo-0613",
        streaming=True,
        callbacks=[StreamCallback()],
    ),  # 모델 정의
    df,  # 데이터프레임
    verbose=True,  # 추론과정 출력
    # AgentType.ZERO_SHOT_REACT_DESCRIPTION
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [39]:
# 질의
agent.invoke({"input": "데이터의 행과 열의 갯수는 어떻게 돼?"})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape'}`


(891, 12)데이터프레임 `df`는 891개의 행과 12개의 열로 구성되어 있습니다.데이터프레임 `df`는 891개의 행과 12개의 열로 구성되어 있습니다.

> Finished chain.


{'input': '데이터의 행과 열의 갯수는 어떻게 돼?',
 'output': '데이터프레임 `df`는 891개의 행과 12개의 열로 구성되어 있습니다.'}

In [38]:
# 질의
agent.run("남자 승객의 생존율을 어떻게 돼? %로 알려줘")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "survival_rate = df[df['Sex'] == 'male']['Survived'].mean() * 100\nsurvival_rate"}`


18.890814558058924남자 승객의 생존율은 약 18.89%입니다.남자 승객의 생존율은 약 18.89%입니다.

> Finished chain.


'남자 승객의 생존율은 약 18.89%입니다.'

In [40]:
# 질의
agent.run(
    "나이가 15세 이하인 승객중 1,2등급에 탑승한 남자 승객의 생존율은 어떻게 돼? %로 알려줘"
)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['Age'] <= 15) & (df['Pclass'].isin([1, 2])) & (df['Sex'] == 'male')]['Survived'].mean() * 100"}`


100.0나이가 15세 이하인 승객 중 1, 2등급에 탑승한 남자 승객의 생존율은 100%입니다.나이가 15세 이하인 승객 중 1, 2등급에 탑승한 남자 승객의 생존율은 100%입니다.

> Finished chain.


'나이가 15세 이하인 승객 중 1, 2등급에 탑승한 남자 승객의 생존율은 100%입니다.'

In [41]:
# 질의
agent.run(
    "Pclass 가 1등급인 승객 중에서 나이가 20세~30세 사이이고, 여성 승객의 생존율은 어떻게 돼? %로 알려줘"
)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['Pclass'] == 1) & (df['Age'] >= 20) & (df['Age'] <= 30) & (df['Sex'] == 'female')]['Survived'].mean() * 100"}`


95.23809523809523Pclass가 1등급인 승객 중에서 나이가 20세~30세 사이이고, 여성 승객의 생존율은 약 95.24%입니다.Pclass가 1등급인 승객 중에서 나이가 20세~30세 사이이고, 여성 승객의 생존율은 약 95.24%입니다.

> Finished chain.


'Pclass가 1등급인 승객 중에서 나이가 20세~30세 사이이고, 여성 승객의 생존율은 약 95.24%입니다.'

## 2개 이상의 DataFrame

2개 이상의 데이터프레임에 기반한 LLM 기반 질의를 할 수 있습니다. 2개 이상의 데이터프레임 입력시 `[]` 로 묶어주면 됩니다.


In [42]:
# 샘플 데이터프레임 생성
df1 = df.copy()
df1 = df1.fillna(0)
df1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S


In [44]:
# 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(
        temperature=0,
        model_name="gpt-3.5-turbo-0613",
        streaming=True,
        callbacks=[StreamCallback()],
    ),
    [df, df1],
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

# 질의
agent.invoke({"input": "나이 컬럼의 나이의 평균차이는 어떻게 돼? %로 구해줘."})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df1['Age'].mean() - df2['Age'].mean()"}`


5.899824717765892나이 컬럼의 나이의 평균 차이는 약 5.9%입니다.나이 컬럼의 나이의 평균 차이는 약 5.9%입니다.

> Finished chain.


{'input': '나이 컬럼의 나이의 평균차이는 어떻게 돼? %로 구해줘.',
 'output': '나이 컬럼의 나이의 평균 차이는 약 5.9%입니다.'}